# Метрики

Здесь собраны метрики и скоры, которые потребовались с в ходе обучения моделей

## Скорбаллы(score, score_point)

### Общая часть

Аналог predict, только переведенная в другую шкалу необычным образом(через логит функцию)

$$
PREDICT = {1 \over (1 + exp(-A*SCORE + B))}
$$

Откуда следует, что, чтобы перевести наши результаты модели, надо

$$
SCORE = {(-ln(1/PREDICT - 1) - B) \over A}
$$

как вычислить A и B? - Берем две нужные нам точки и решаем систему уравнений(например что при PREDICT_1 = 0.001 SCORE_1 = 1400, а при PREDICT_2 = 0.99 SCORE_2 = 100)

а также ниже написана функция в которую нужно просто внести значения, а значения A и B она посчитает за вас

*на вход:*

1. predict_1 - предикт первой точки
2. score_1 - необходимый скор балл первой точки
3. predict_2 - предикт второй точки
4. score_2 - необходимый скор балл второй точки

*на выходе:*

коэффициенты A и B которые надо подставить в формулу для расчета SCORE


###  Реализация

https://gitlab.rgsbank.ru/a.kaznacheev_dev/module_risk_kb/-/blob/master/model_engineering/metrics.py

In [3]:
from module_risk_kb.model_engineering import metrics

metrics.get_A_B_coeff_for__score_point(0.001,1400,0.99,100)

A =  -0.008847595868294722
B =  5.4798794369640556


# Оценка возможности разбиения на интервалы

на вход поступает массив вида:

In [47]:
pd.DataFrame({'predict':[0.5,0.1,0.1,0.2,0.3,0.2,0.8],'target':[1,0,1,0,1,1,1]})

predict  target
0      0.5       1
1      0.1       0
2      0.1       1
3      0.2       0
4      0.3       1
5      0.2       1
6      0.8       1

## Фиксированные интервалы

### Общая часть

*на вход:*

1. predict_target - датафрейм в виде двух столбцов
2. thresholds_min - нижние границы интервалов(необязательный парамет)
3. thresholds_max - верхние границы интервалов(необязательный парамет)

*на выходе:*

интервалы и количество единиц и нулей в них


###  Реализация

https://gitlab.rgsbank.ru/a.kaznacheev_dev/module_risk_kb/-/blob/master/model_engineering/metrics.py

In [6]:
import pandas as pd
from module_risk_kb.model_engineering import metrics

thresholds_min = [0,      0.01,  0.2,  0.5]
thresholds_max = [0.01, 0.2,   0.5,    1]

predict_target = pd.DataFrame({'predict':[0.5,0.1,0.1,0.2,0.3,0.2,0.8],'target':[1,0,1,0,1,1,1]})

metrics.get_fixed_intervals_quality(predict_target, thresholds_min=thresholds_min,thresholds_max=thresholds_max)

0    1  1/(0+1) (%)  interval  max predict  mean predict  min predict
0  0.0  0.0     0.000000    0-0.01          NaN           NaN          NaN
1  1.0  1.0    50.000000  0.01-0.2          0.1      0.100000          0.1
2  1.0  2.0    66.666667   0.2-0.5          0.3      0.233333          0.2
3  0.0  2.0   100.000000     0.5-1          0.8      0.650000          0.5

## Плавающие интервалы

### Общая часть

здесь две функции:

1. присваивает группу строке
2. группирует по группам

*на вход:*

1. predict_target - датафрейм в виде двух столбцов
2. number_of_group - количество групп(по умолчанию 10)

*на выходе:*

интервалы и количество единиц и нулей в них

###  Реализация

https://gitlab.rgsbank.ru/a.kaznacheev_dev/module_risk_kb/-/blob/master/model_engineering/metrics.py

In [10]:
import pandas as pd
from module_risk_kb.model_engineering import metrics

predict_target = pd.DataFrame({'predict':[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.9,0.9,0.9,0.9,0.1,0.1,0.001,0.01,0.01],'target':[1,1,0,1,0,1,0,1,1,1,1,1,1,1,0,0,0,0,0]})

result_no_fixed = metrics.no_fixed_intervals_quality(predict_target, number_of_group=5)
metrics.get_no_fixed_statistics(result_no_fixed)

mean_target  mean_predict  min_predict  max_predict  1  0  \
group_number                                                              
0                0.333333      0.091222          0.0          0.3  3  6   
1                0.500000      0.550000          0.4          0.7  2  2   
2                1.000000      0.850000          0.8          0.9  2  0   
3                1.000000      0.900000          0.9          0.9  2  0   
4                1.000000      0.900000          0.9          0.9  2  0   

              1/(0+1) (%)  
group_number               
0               33.333333  
1               50.000000  
2              100.000000  
3              100.000000  
4              100.000000